# Ponder and Prove Data Compression
#### Due: Saturday, 6 March 2021, 11:59 pm.

# DONE Explore Huffman Trees and Huffman Codes


Your task is examine how to compress a *special piece of information* as compactly as possible, and **calculate various compression ratios**.

Recall that the **compression ratio** of a variable-length encoding like Huffman encoding is the percentage $100(f - v)/f$, where $f$ is the number of bits per symbol of the smallest **fixed**-length encoding, and $v$ is the average number of bits per symbol with the variable-length encoding.

For example, if there were 9 different symbols in a message, $f=4$ is the number of bits of the smallest fixed-length encoding, because $2^3 = 8$ (not enough for $9$) and $2^4 = 16$ (enough and to spare). If the variable-length encoding of the message had $v=3.12$, the compression ratio would be $100(4 - 3.12)/4 \approx 22\%$.

Note that calculating the average number of bits per symbol is not strictly necessary. That's because an alternate and equivalent way is to calculate $100(ft - vt)/ft$, where $ft$ is the **total** number of bits encoded with the fixed encoding, and $vt$ is the **total** number of bits encoded with the variable-length encoding.

The *special piece of information* to be compressed is a list of the first ten million primes. This is a list that starts

|    |
|----|
|  2 |
|  3 |
|  5 |
|  7 |
| 11 |
| 13 |
| 17 |
| 19 |
| 23 |
| 29 |

  and ends

|           |
|-----------|
| 179424551 |
| 179424571 |
| 179424577 |
| 179424601 |
| 179424611 |
| 179424617 |
| 179424629 |
| 179424667 |
| 179424671 |
| 179424673 |

As ASCII text stored in a file with one prime per line, the size of this data file is slightly over 89 megabytes. The goal is to compress this down to just over 5 megabytes (5589056 bytes, to be exact). That's a 94% compression ratio!

Standard compression tools can only get about a 73% compression ratio for this ASCII data. A more clever approach is needed. Instead of compressing the list of prime numbers, compress a list of the *gaps* between them!

It doesn't save much, just the unique (occurring only once) gap size of 1 between 2 and 3, but in the spirit of de Polignac's conjecture that every *even* number appears infinitely often as a gap between consecutive primes, just consider the even-sized gaps. The result will be a list that starts with 2 (the difference between 5 and 3), 2 (the difference between 7 and 5), 4 (the difference between 11 and 7), 2 (the difference between 13 and 11), 4 (the difference between 17 and 13), 2 (the difference between 19 and 17), 4 (the difference between 23 and 19), and 6 (the difference between 29 and 23).

Generating this data is the first task. The algorithm for doing so is very straightforward:

1. Find the gaps between consecutive odd primes.
2. Store these gaps as a list of even numbers.

Tabulating the results, the first ten gaps and the last ten gaps are as follows, where the numbers after the equals signs are the gaps to list:

|                 |
|-----------------|
|  5  -   3  =  2 |
|  7  -   5  =  2 |
| 11  -   7  =  4 |
| 13  -  11  =  2 |
| 17  -  13  =  4 |
| 19  -  17  =  2 |
| 23  -  19  =  4 |
| 29  -  23  =  6 |
| 31  -  29  =  2 |
| 37  -  31  =  6 |

|                                |
|--------------------------------|
| 179424551  -  179424533  =  18 |
| 179424571  -  179424551  =  20 |
| 179424577  -  179424571  =   6 |
| 179424601  -  179424577  =  24 |
| 179424611  -  179424601  =  10 |
| 179424617  -  179424611  =   6 |
| 179424629  -  179424617  =  12 |
| 179424667  -  179424629  =  38 |
| 179424671  -  179424667  =   4 |
| 179424673  -  179424671  =   2 |

As a correctness check, see if your generated list of gaps has length 9999998.

The next step is to count how many times each gap size occurs, so that for the Huffman encoding scheme, the larger the frequency of occurrence, the smaller the number of bits encoding that gap size.

As a correctness check, here are the first ten and the last ten gap counts:

|  Gap | Count   |
|------|---------|
|    2 |  738597 |
|    4 |  738717 |
|    6 | 1297540 |
|    8 |  566151 |
|   10 |  729808 |
|   12 |  920661 |
|   14 |  503524 |
|   16 |  371677 |
|   18 |  667734 |
|   20 |  354267 |
|      |         |
|  190 |       1 |
|  192 |       3 |
|  194 |       1 |
|  196 |       1 |
|  198 |       6 |
|  202 |       2 |
|  204 |       3 |
|  210 |       4 |
|  220 |       1 |
|  222 |       1 |

Note two things from these partial gap counts:

1. Small even numbers (< 100) are well represented, larger ones (< 1000) less so.
2. Ten million primes aren't enough to have *every* even number represented; for example, 200, 206, 208, 212, 214, 216, and 218 do not appear even once.


# DONE Determine Exact Size of Data to be Compressed


Without actually doing it, imagine creating an ASCII file containing the first ten million primes, represented in decimal, one prime per line. Calculate the size of this file, so you can show an exceptional compression ratio from it (see below).

Using a binary encoding instead of ASCII, each prime requires 32 bits (4 bytes), so the size of a binary file is easily determined.

Using a fixed-width encoding of the gap counts, however, requires knowing how many different gap sizes there are, after which the calculation is straightforward.

Answer

The below code calculates that the ASCII file of 10,000,000 primes would be about 93 MB. That's a bit more than is said up above, but it's pretty close.

The binary file would just be 40 MB because each number is simply 4 bytes.

In [2]:
#!pip install pyprimesieve
import pyprimesieve
tmp = pyprimesieve.primes_nth(10000000)
primes = pyprimesieve.primes(tmp+1)
fileLength = 0
for prime in primes:
  prime = str(prime)
  fileLength += len(prime) + 1

print(fileLength)



93484450


# DONE Use Functional Python


You are encouraged to use the [anytree](https://pypi.org/project/anytree) Python library, which has a nice exporter by way of which you can graphically view trees. (You may recall using this in DM1, and thus know that **anytree** depends on [graphviz](https://graphviz.org), which you also used.)

This library uses the object-oriented features of Python to create and visualize trees. You are encouraged to use the functional features of Python as much as possible, achieving your results not by using some existing third-party libraries for building Huffman Trees and Codes, but writing your own code as cleanly and elegantly as you can.

In [3]:
!pip install anytree

     |████████████████████████████████| 51kB 2.7MB/s 


# TODO Achieve Target Compression Ratios

Your solution should correctly compute the following three compression ratios:

| Ratio       | Value              |
|-------------|--------------------|
| From fixed  | 36.125168653605158 |
| From binary |              86.03 |
| From ASCII  |              94.02 | 

The below code shows my best shot at this. It doesn't give the right answer, but it does give an answer. It took my a long time to write this code, and I'm not sure why I'm not getting the same answer.


In [74]:
from anytree import Node, RenderTree, LevelOrderGroupIter
from collections import Counter
from math import ceil, log

def show_results(message, code_tuples):
  total_characters = len(message)
  total_unique_characters = len(code_tuples)
  total_bits = 0
  for char, count, code in code_tuples:
    total_bits += count * len(code)
  average_bits_per_character = total_bits / total_characters
  fixed_bits_per_character = ceil(log(total_unique_characters, 2))
  total_fixed_bits = total_characters * fixed_bits_per_character
  compression_ratio = (total_fixed_bits - total_bits) / total_fixed_bits
  print(f'          Total Characters: {total_characters}')
  print(f'   Total Unique Characters: {total_unique_characters}')
  print(f'                Total Bits: {total_bits}')
  print(f'Average Bits per Character: {average_bits_per_character:.2f}')
  print(f'  Fixed Bits per Character: {fixed_bits_per_character}')
  print(f'          Total Fixed Bits: {total_fixed_bits}')
  print(f'         Compression Ratio: {compression_ratio:.3f}')

tmp = pyprimesieve.primes_nth(10000000)
primes = pyprimesieve.primes(tmp+1)
primes_string = ''
for prime in primes:
 primes_string += str(prime) + ' '
gaps = [*map(lambda i:primes[i]-primes[i-1],range(1,10000000))]

gapsString = ''
for gap in gaps:
  gapsString += str(gap) + ' '

counterGaps = Counter(gapsString)
print(counterGaps)

Counter({' ': 9999999, '2': 3890666, '1': 3212887, '4': 2479713, '6': 2354475, '0': 1745468, '8': 1741680, '3': 953233, '5': 215614, '7': 64336, '9': 16760})


In [64]:
gap_leaves = []
for count in counterGaps.most_common():
  gap_leaves.append(Node(count[0], num=count[1], code=''))
  print(gap_leaves[-1].num)

9999999
3890666
3212887
2479713
2354475
1745468
1741680
953233
215614
64336
16760


In [65]:
def nodeNum(node):
  return node.num
gap_leaves.sort(key=nodeNum)
gap_nodes = gap_leaves.copy()

while(len(gap_nodes) > 1):
  newNode = Node(gap_nodes[0].name + gap_nodes[1].name, num=(gap_nodes[0].num + gap_nodes[1].num), code='', children=gap_nodes[:2])
  gap_nodes[0].parent = newNode
  print(gap_nodes[0].parent)
  gap_nodes[1].parent = newNode
  #print(RenderTree(newNode))
  gap_nodes.pop(0)
  gap_nodes.pop(0)
  gap_nodes.append(newNode)
  gap_nodes.sort(key=nodeNum)

print(RenderTree(gap_nodes[0]))


Node('/97', code='', num=81096)
Node('/975', code='', num=296710)
Node('/9753', code='', num=1249943)
Node('/97538', code='', num=2991623)
Node('/06', code='', num=4099943)
Node('/497538', code='', num=5471336)
Node('/12', code='', num=7103553)
Node('/06497538', code='', num=9571279)
Node('/1206497538', code='', num=16674832)
Node('/ 1206497538', code='', num=26674831)
Node('/ 1206497538', code='', num=26674831)
├── Node('/ 1206497538/ ', code='', num=9999999)
└── Node('/ 1206497538/1206497538', code='', num=16674832)
    ├── Node('/ 1206497538/1206497538/12', code='', num=7103553)
    │   ├── Node('/ 1206497538/1206497538/12/1', code='', num=3212887)
    │   └── Node('/ 1206497538/1206497538/12/2', code='', num=3890666)
    └── Node('/ 1206497538/1206497538/06497538', code='', num=9571279)
        ├── Node('/ 1206497538/1206497538/06497538/06', code='', num=4099943)
        │   ├── Node('/ 1206497538/1206497538/06497538/06/0', code='', num=1745468)
        │   └── Node('/ 1206497538/1

In [67]:
for level in LevelOrderGroupIter(gap_nodes[0]):
  print(level)
  for i in range(len(level)):
    level[i].code = (level[i].parent.code if level[i].parent != None else '') + str(i % 2)
  
print(gap_leaves)

(Node('/ 1206497538', code='', num=26674831),)
(Node('/ 1206497538/ ', code='', num=9999999), Node('/ 1206497538/1206497538', code='', num=16674832))
(Node('/ 1206497538/1206497538/12', code='', num=7103553), Node('/ 1206497538/1206497538/06497538', code='', num=9571279))
(Node('/ 1206497538/1206497538/12/1', code='', num=3212887), Node('/ 1206497538/1206497538/12/2', code='', num=3890666), Node('/ 1206497538/1206497538/06497538/06', code='', num=4099943), Node('/ 1206497538/1206497538/06497538/497538', code='', num=5471336))
(Node('/ 1206497538/1206497538/06497538/06/0', code='', num=1745468), Node('/ 1206497538/1206497538/06497538/06/6', code='', num=2354475), Node('/ 1206497538/1206497538/06497538/497538/4', code='', num=2479713), Node('/ 1206497538/1206497538/06497538/497538/97538', code='', num=2991623))
(Node('/ 1206497538/1206497538/06497538/497538/97538/9753', code='', num=1249943), Node('/ 1206497538/1206497538/06497538/497538/97538/8', code='', num=1741680))
(Node('/ 12064975

In [69]:
code_tuples = []
for leaf in gap_leaves:
  print(leaf.name[-1] + ' : ' + leaf.code)
  code_tuples.append((leaf.name[-1], leaf.num, leaf.code))

9 : 011110000
7 : 011110001
5 : 01111001
3 : 0111101
8 : 011111
0 : 01100
6 : 01101
4 : 01110
1 : 0100
2 : 0101
  : 00


In [75]:
show_results(primes_string, code_tuples)

          Total Characters: 93484450
   Total Unique Characters: 11
                Total Bits: 100889977
Average Bits per Character: 1.08
  Fixed Bits per Character: 4
          Total Fixed Bits: 373937800
         Compression Ratio: 0.730


# TODO My Report on What I Did and What I Learned

## Fun


I would say this has been a fun Ponder and Prove. I spent my first hour working with Alex, and that went well. We were both kind of confused, but we helped each other get started. I had fun learning more about trees and how to code them in Python. It was a good challange.

## New

I learned a lot about coding with Anytree. It was fun learning how to make code that would find Huffman trees. I started with no idea what I was doing, but now I have some idea of what I did.

## Meaningful


I could build upon my know knowledge of coding with trees in Python. They are a useful data structure. I'm sure I'll use them again.

## Other

I worked with Alex Vasiuk for the first part of this. We would have worked more together, but he ran out of time. It's been a good experience. I shared some code with him too.

Also, I just realized that I didn't do this quite right. I was supposed to code each of the number values in the gap sizes, not the digits. That's my bad. I see why I'm not getting the same values.

# TODO What is True?
Click on each warranted checkbox to toggle it to True (or back to False). 

NOTE: *This only works in Colab. If you run it in some other Jupyter notebook client/server environment you may have to change False to True (or vice versa) manually.*

This self-assessment is subject to revision by a grader.

In [ ]:
#@markdown ## What is True about what I did?
#@markdown ### I had fun.
cb00 = True #@param {type:'boolean'}
#@markdown ### I learned something new.
cb01 = True #@param {type:'boolean'}
#@markdown ### I achieved something meaningful, or something I can build upon at a later time.
cb02 = True #@param {type:'boolean'}
#@markdown ## What is True about my report?
#@markdown ### I wrote a sufficient number of well-written sentences.
cb03 = True #@param {type:'boolean'}
#@markdown ### My report is free of mechanical infelicities.
cb04 = True #@param {type:'boolean'}
#@markdown ### I used Grammarly (or something better described in my report) to check for MIs.
cb05 = False #@param {type:'boolean'}
#@markdown ### I reported on any connections I found between these problems and something I already know. 
cb06 = False #@param {type:'boolean'}
#@markdown ### I reported who were and what contribution each of my collaborators made.
cb07 = True #@param {type:'boolean'}
#@markdown ## What is True about my calculations?
#@markdown ### I correctly calculated the number of times each gap size occurs. 
cb08 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the number of bits per gap size with a fixed encoding.
cb09 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the Huffman encoding.
cb10 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the fixed encoding.
cb11 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from this fixed encoding.
cb12 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as 32-bit integer binary data.
cb13 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the binary size.
cb14 = False #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as ASCII data.
cb15 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the ASCII size (just the primes, nothing else).
cb16 = False #@param {type:'boolean'}

## DO NOT CHANGE ANYTHING IN THE NEXT CODE CELL!!
### Delete this cell and the following ones before submitting your work.

In [ ]:
points_for_what_I_did = [5]*3
points_for_my_report = [8]*5
points_for_my_calculations = [5]*9
points = points_for_what_I_did + points_for_my_report + points_for_my_calculations
# cb is short for checkbox
total = sum(map(lambda n, p: p if eval(f'cb{n:02}') else 0,
                range(len(points)), points))             
total

# For graders

In [ ]:
#@markdown ---
number_of_MIs_found = 0 #@param {type: 'slider', min: 0, max: 5}
#@markdown ---
